In [47]:
# import librerie
import os
import tweepy
import facebook
import requests
import datetime
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import json
import requests

In [48]:
# configuration file
config = {}
config_path = os.path.join(os.path.abspath('../../'))
config_name = 'config.py'
config_file = os.path.join(config_path,config_name)
exec(open(config_file).read(),config)
nw_key=config['TOKEN_NW']

In [49]:
# Candidati Elezioni
users = [
    {'user':config['USER1'],'user_id':config['USER1_ID_TW']},
    {'user':config['USER2'],'user_id':config['USER2_ID_TW']},
    {'user':config['USER3'],'user_id':config['USER3_ID_TW']}
]

In [50]:
# get today's date
todays_date = datetime.datetime.now()
yesterday = datetime.datetime.now() - datetime.timedelta(days=1)
str_dt = str(yesterday.date())

In [51]:
def get_user_news(user,dt,today,key):
    url = ('https://newsapi.org/v2/everything?'
       'q='+user+'&'
       'from='+dt+'&'
       'sortBy=publishedAt&'
       'language=it&'
       'apiKey='+key)
    response = requests.get(url)
    json_data = json.loads(response.text)
    articles = json_data['articles']
    l_article = []
    for art in articles[0:5]:
        d_article = {
            'user':'all',
            'autore':art['author'],
            'desc':art['description'],
            'pubAt':art['publishedAt'],
            'fonte':art['source']['name'],
            'titolo':art['title'],
            'url':art['url'],
            'img':art['urlToImage'],
            'dt_rif':today
        }
        l_article.append(d_article)
    
    return l_article

In [52]:
l = []
for user in users:
    l.append(get_user_news(user['user'],str_dt,todays_date,nw_key))

In [53]:
df0 = pd.DataFrame(l[0])
df1 = pd.DataFrame(l[1])
df2 = pd.DataFrame(l[2])

In [54]:
df = df0.append(df1).append(df2)

In [55]:
df.drop_duplicates(inplace=True)

In [56]:
df.count()

autore    7
desc      8
dt_rif    8
fonte     8
img       8
pubAt     8
titolo    8
url       8
user      8
dtype: int64

In [58]:
df.head(2)

,autore,desc,dt_rif,fonte,img,pubAt,titolo,url,user
0,Gisella Ruccia,"Matteo Renzi chiude la Leopolda di Firenze, in...",2017-11-26 20:26:32.851152,Ilfattoquotidiano.it,https://st.ilfattoquotidiano.it/wp-content/upl...,2017-11-26T18:48:03Z,"Leopolda, Renzi: ‘Basta litigi, no rancore’. F...",https://www.ilfattoquotidiano.it/2017/11/26/le...,all
1,None,Da www.ansa.it matteo renzi leopolda 2017 Matt...,2017-11-26 20:26:32.851152,Dagospia.com,http://cdn-static.dagospia.com/img/patch/11-20...,2017-11-26T17:23:51Z,"QUANDO NON SAI CHE FARE, VAI CON IL CAVALLO DI...",http://www.dagospia.com/rubrica-3/politica/qua...,all


In [59]:
# get database connection
db=config['DATABASE_ELE']
schema=config['SCHEMA_ELE']
engine = create_engine(db)

In [60]:
# write on db
df.to_sql('news', engine, schema=schema, if_exists='append')